# Q1

In [64]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

col_names = ["duration","protocol_type","service","flag","src_bytes",
"dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
"logged_in","num_compromised","root_shell","su_attempted","num_root",
"num_file_creations","num_shells","num_access_files","num_outbound_cmds",
"is_host_login","is_guest_login","count","srv_count","serror_rate",
"srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
"diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
"dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","class","difficulty"]

nominal_cols = ['protocol_type','service','flag']
binary_cols = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login',
'is_guest_login']
continuous_cols = ['duration' ,'src_bytes', 'dst_bytes', 'wrong_fragment' ,'urgent', 'hot',
'num_failed_logins', 'num_compromised', 'num_root' ,'num_file_creations',
'num_shells', 'num_access_files', 'num_outbound_cmds', 'count' ,'srv_count',
'serror_rate', 'srv_serror_rate' ,'rerror_rate' ,'srv_rerror_rate',
'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate' ,'dst_host_count',
'dst_host_srv_count' ,'dst_host_same_srv_rate' ,'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate' ,'dst_host_srv_diff_host_rate',
'dst_host_serror_rate' ,'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate']

class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        dos = ['apache2', 'back', 'land', 'neptune', 
               'mailbomb', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm', 'worm']
        probe = ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']
        u2r = ['buffer_overflow', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm']
        r2l = ['ftp_write', 'guess_passwd', 'httptunnel', 'imap', 'multihop', 'named', 
               'phf', 'sendmail', 'snmpgetattack', 'spy', 'snmpguess', 'warezclient', 
               'warezmaster', 'xlock', 'xsnoop']
        label_to_binary = udf(lambda name: 0.0 if name == 'normal' else 
                              (1.0 if name in dos else 
                               (2.0 if name in probe else 
                                (3.0 if name in u2r else 4.0))))
        output_df = dataset.withColumn('outcome', label_to_binary(col('class'))).drop("class")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        output_df = output_df.drop('difficulty')
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+binary_cols+nominal_onehot_cols
    corelated_cols_to_remove = ["dst_host_serror_rate","srv_serror_rate","dst_host_srv_serror_rate",
                     "srv_rerror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate"]
    for col_name in corelated_cols_to_remove:
        feature_cols.remove(col_name)
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols+nominal_id_cols+
        nominal_onehot_cols+ binary_cols + continuous_cols + ['vectorized_features'])
    

    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    
    return pipeline 

In [65]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()

nslkdd_raw = spark.read.csv('../NSL-KDD/KDDTrain+.txt',header=False).toDF(*col_names)
nslkdd_test_raw = spark.read.csv('../NSL-KDD/KDDTest+.txt',header=False).toDF(*col_names)

preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(nslkdd_raw)

nslkdd_df = preprocess_pipeline_model.transform(nslkdd_raw)
nslkdd_df_test = preprocess_pipeline_model.transform(nslkdd_test_raw)


In [33]:
nslkdd_df.groupBy('outcome').count().show()

+-------+-----+
|outcome|count|
+-------+-----+
|    0.0|67343|
|    1.0|45927|
|    4.0|  995|
|    3.0|   52|
|    2.0|11656|
+-------+-----+



In [66]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

# Creating the training, validation and test data sets
nslkdd_df_train = nslkdd_df 
nslkdd_df_validate,nslkdd_df_test = nslkdd_df_test.randomSplit([0.5,0.5])

nslkdd_df_train_pandas = nslkdd_df_train.withColumn('features', to_array('features')).toPandas()
nslkdd_df_validate_pandas = nslkdd_df_validate.withColumn('features', to_array('features')).toPandas()
nslkdd_df_test_pandas = nslkdd_df_test.withColumn('features', to_array('features')).toPandas()

In [199]:
import tensorflow as tf
from tensorflow import keras 

# Converting the pandas DataFrame to tensors

x_train = tf.constant(np.array(nslkdd_df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(nslkdd_df_train_pandas['outcome'].values.tolist()))

x_validate = tf.constant(np.array(nslkdd_df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(nslkdd_df_validate_pandas['outcome'].values.tolist()))


x_test = tf.constant(np.array(nslkdd_df_test_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(nslkdd_df_test_pandas['outcome'].values.tolist()))


In [200]:
# Defining the model with 5 hidden layers and the output layer for 5 classes

model = keras.Sequential( [keras.layers.Dense(10,activation='relu'),
                           keras.layers.Dense(100,activation='relu'),
                           keras.layers.Dense(100,activation='relu'),
                           keras.layers.Dense(10,activation='relu') ,
                           keras.layers.Dense(5, activation='softmax')] ) # Added softmax activation, output will be probability


y_pred = model(x_train)
model.summary()



Model: "sequential_79"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_461 (Dense)           (125973, 10)              1140      
                                                                 
 dense_462 (Dense)           (125973, 100)             1100      
                                                                 
 dense_463 (Dense)           (125973, 100)             10100     
                                                                 
 dense_464 (Dense)           (125973, 10)              1010      
                                                                 
 dense_465 (Dense)           (125973, 5)               55        
                                                                 
Total params: 13,405
Trainable params: 13,405
Non-trainable params: 0
_________________________________________________________________


In [201]:
# Compile the model
model.compile(optimizer = 'sgd',
    loss=keras.losses.SparseCategoricalCrossentropy(), # Used this loss function due to the format of the output.
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [202]:
import datetime

# This is to send the model output to tensorboard for visualization.
log_dir = "logsHW7/Q1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [203]:
# Fitting the model

model.fit(x_train,y_train, epochs = 10,validation_data=(x_validate,y_validate),verbose = 2,
          callbacks=[tensorboard_callback])

Epoch 1/10


2022-11-10 19:56:51.004165: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-10 19:57:15.818486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3937/3937 - 27s - loss: 0.1715 - sparse_categorical_accuracy: 0.9545 - val_loss: 1.4537 - val_sparse_categorical_accuracy: 0.7437 - 27s/epoch - 7ms/step
Epoch 2/10
3937/3937 - 26s - loss: 0.0567 - sparse_categorical_accuracy: 0.9819 - val_loss: 1.9440 - val_sparse_categorical_accuracy: 0.7571 - 26s/epoch - 7ms/step
Epoch 3/10
3937/3937 - 26s - loss: 0.0405 - sparse_categorical_accuracy: 0.9871 - val_loss: 2.0554 - val_sparse_categorical_accuracy: 0.7566 - 26s/epoch - 7ms/step
Epoch 4/10
3937/3937 - 27s - loss: 0.0351 - sparse_categorical_accuracy: 0.9887 - val_loss: 2.1886 - val_sparse_categorical_accuracy: 0.7516 - 27s/epoch - 7ms/step
Epoch 5/10
3937/3937 - 26s - loss: 0.0328 - sparse_categorical_accuracy: 0.9893 - val_loss: 2.1320 - val_sparse_categorical_accuracy: 0.7527 - 26s/epoch - 7ms/step
Epoch 6/10
3937/3937 - 27s - loss: 0.0313 - sparse_categorical_accuracy: 0.9902 - val_loss: 2.2020 - val_sparse_categorical_accuracy: 0.7507 - 27s/epoch - 7ms/step
Epoch 7/10
3937/3937 - 26s 

In [204]:
# Evaluating the model

model.evaluate(x_test,y_test, verbose = 2)

354/354 - 3s - loss: 2.2163 - sparse_categorical_accuracy: 0.7614 - 3s/epoch - 8ms/step


[2.216282367706299, 0.7614410519599915]

# Q2

In [190]:
from tensorboard.plugins.hparams import api as hp

# Defining the hyperparameters that will be compared and tuned using tensorboard

HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20,30])) # the width of the layers
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([4,6])) # Number of layers/depth
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd'])) # Using different optimizers

# Creating the logs
with tf.summary.create_file_writer('logsHW7/Q2').as_default():
  hp.hparams_config(
    hparams=[HP_WIDTH, HP_DEPTH, HP_OPTIMIZER],
    metrics=[hp.Metric('Accuracy')],
  )



In [191]:
# Defining the model training function for hyperparameter tuning
def train_test_model(hparams,logdir, x, y, i, k):
  model = keras.Sequential()
  for _ in range(hparams[HP_DEPTH]): #Create the model
    model.add(keras.layers.Dense(hparams[HP_WIDTH],activation='relu'))
  model.add(keras.layers.Dense(5, activation = 'softmax'))
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss = keras.losses.SparseCategoricalCrossentropy(),
      metrics=[keras.metrics.SparseCategoricalAccuracy(name = "Accuracy_epochs")])
    
  # This section is for splitting the data sets into k folds. May be better to do this outside this function
  x_train = tf.split(x, k)
  y_train = tf.split(y, k)
  val_x = x_train.pop(i)
  val_y = y_train.pop(i)
  x_train = tf.concat([x_train[0], x_train[1]], 0)  
  y_train = tf.concat([y_train[0], y_train[1]], 0)  

  # Store the output of the model training
  history = model.fit(x_train, y_train, epochs=5, verbose = 2,
  callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)],
  validation_data = (val_x, val_y))

  accuracy = np.max(history.history["val_Accuracy_epochs"]) # Returns the highest accuracy obtained in training
  return accuracy


In [197]:
# Combine the original training and validation data sets
x_train = tf.concat([x_train,x_validate], 0)
y_train = tf.concat([y_train,y_validate], 0)

# Get randomized indices in order to shuffle data sets
indices = tf.range(start=0, limit=tf.shape(x_train)[0], dtype=tf.int32)

shuffled_indices = tf.random.shuffle(indices)

# Shuffle the data sets with the random indices
x_train_shuffled = tf.gather(x_train, shuffled_indices)
y_train_shuffled = tf.gather(y_train, shuffled_indices)
print(tf.shape(x_train_shuffled))

# Number of folds to split the data into
k = 3

# Splits the data into k folds. These are unused as splitting is done also within the train_test_model function.
x_train_split = tf.split(x_train_shuffled, k)
y_train_split = tf.split(y_train_shuffled, k)

print(tf.shape(x_train_split))
print(tf.shape(y_train_split))

tf.Tensor([137220    113], shape=(2,), dtype=int32)
tf.Tensor([    3 45740   113], shape=(3,), dtype=int32)
tf.Tensor([    3 45740], shape=(2,), dtype=int32)


In [198]:
# Nested loops for hyperparameter tuning, iterating over all the hyperparameters and combinations
for hp_width in HP_WIDTH.domain.values:
  for hp_depth in (HP_DEPTH.domain.values):
    for hp_optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_WIDTH: hp_width,
            HP_DEPTH: hp_depth,
            HP_OPTIMIZER: hp_optimizer,
        }
        run_name = f"run-OPTIMIZER{hparams[HP_OPTIMIZER]}-WIDTH{int(hparams[HP_WIDTH])}-DEPTH{hparams[HP_DEPTH]}"
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})

        run_dir = 'logsHW7/Q2/' + run_name
        
        # The model is trained with each of the k folds in the dataset and accuracy is averaged over the k runs.
        accuracy = 0
        for i in range(k):
            accuracy += train_test_model(hparams, run_dir, x_train_shuffled, y_train_shuffled, i, k)
        accuracy = accuracy / k

        with tf.summary.create_file_writer(run_dir).as_default():
          hp.hparams(hparams)  # record the values used in this trial
          tf.summary.scalar("Accuracy", accuracy, step=1)

    

--- Starting trial: run-OPTIMIZERadam-WIDTH20-DEPTH4
{'NN_width': 20, 'NN_depth': 4, 'optimizer': 'adam'}
Epoch 1/5


2022-11-10 08:34:04.073400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-10 08:34:24.644616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2859/2859 - 27s - loss: 0.1250 - Accuracy_epochs: 0.9642 - val_loss: 0.0693 - val_Accuracy_epochs: 0.9787 - 27s/epoch - 10ms/step
Epoch 2/5
2859/2859 - 26s - loss: 0.0564 - Accuracy_epochs: 0.9825 - val_loss: 0.0632 - val_Accuracy_epochs: 0.9818 - 26s/epoch - 9ms/step
Epoch 3/5
2859/2859 - 26s - loss: 0.0475 - Accuracy_epochs: 0.9850 - val_loss: 0.0497 - val_Accuracy_epochs: 0.9844 - 26s/epoch - 9ms/step
Epoch 4/5
2859/2859 - 26s - loss: 0.0424 - Accuracy_epochs: 0.9863 - val_loss: 0.0438 - val_Accuracy_epochs: 0.9861 - 26s/epoch - 9ms/step
Epoch 5/5
2859/2859 - 27s - loss: 0.0386 - Accuracy_epochs: 0.9878 - val_loss: 0.0436 - val_Accuracy_epochs: 0.9858 - 27s/epoch - 10ms/step
Epoch 1/5


2022-11-10 08:36:17.122226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-10 08:36:41.029582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2859/2859 - 32s - loss: 0.1234 - Accuracy_epochs: 0.9634 - val_loss: 0.0631 - val_Accuracy_epochs: 0.9813 - 32s/epoch - 11ms/step
Epoch 2/5
2859/2859 - 27s - loss: 0.0574 - Accuracy_epochs: 0.9826 - val_loss: 0.0557 - val_Accuracy_epochs: 0.9831 - 27s/epoch - 9ms/step
Epoch 3/5
2859/2859 - 26s - loss: 0.0497 - Accuracy_epochs: 0.9847 - val_loss: 0.0476 - val_Accuracy_epochs: 0.9839 - 26s/epoch - 9ms/step
Epoch 4/5
2859/2859 - 26s - loss: 0.0446 - Accuracy_epochs: 0.9859 - val_loss: 0.0447 - val_Accuracy_epochs: 0.9861 - 26s/epoch - 9ms/step
Epoch 5/5
2859/2859 - 26s - loss: 0.0409 - Accuracy_epochs: 0.9868 - val_loss: 0.0453 - val_Accuracy_epochs: 0.9857 - 26s/epoch - 9ms/step
Epoch 1/5


...

2859/2859 - 27s - loss: 0.2281 - Accuracy_epochs: 0.9316 - val_loss: 0.1024 - val_Accuracy_epochs: 0.9673 - 27s/epoch - 10ms/step
Epoch 2/5
2859/2859 - 27s - loss: 0.0828 - Accuracy_epochs: 0.9746 - val_loss: 0.0740 - val_Accuracy_epochs: 0.9762 - 27s/epoch - 9ms/step
Epoch 3/5
2859/2859 - 27s - loss: 0.0656 - Accuracy_epochs: 0.9793 - val_loss: 0.0669 - val_Accuracy_epochs: 0.9792 - 27s/epoch - 9ms/step
Epoch 4/5
2859/2859 - 26s - loss: 0.0572 - Accuracy_epochs: 0.9816 - val_loss: 0.0722 - val_Accuracy_epochs: 0.9761 - 26s/epoch - 9ms/step
Epoch 5/5
2859/2859 - 27s - loss: 0.0518 - Accuracy_epochs: 0.9830 - val_loss: 0.0559 - val_Accuracy_epochs: 0.9833 - 27s/epoch - 9ms/step
Epoch 1/5


2022-11-10 09:23:43.502353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-10 09:24:03.875568: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2859/2859 - 28s - loss: 0.2226 - Accuracy_epochs: 0.9320 - val_loss: 0.1033 - val_Accuracy_epochs: 0.9669 - 28s/epoch - 10ms/step
Epoch 2/5
2859/2859 - 27s - loss: 0.0846 - Accuracy_epochs: 0.9742 - val_loss: 0.0810 - val_Accuracy_epochs: 0.9773 - 27s/epoch - 10ms/step
Epoch 3/5
2859/2859 - 32s - loss: 0.0662 - Accuracy_epochs: 0.9801 - val_loss: 0.0647 - val_Accuracy_epochs: 0.9795 - 32s/epoch - 11ms/step
Epoch 4/5
2859/2859 - 27s - loss: 0.0579 - Accuracy_epochs: 0.9824 - val_loss: 0.0553 - val_Accuracy_epochs: 0.9821 - 27s/epoch - 10ms/step
Epoch 5/5
2859/2859 - 30s - loss: 0.0528 - Accuracy_epochs: 0.9836 - val_loss: 0.0582 - val_Accuracy_epochs: 0.9828 - 30s/epoch - 10ms/step
Epoch 1/5


2022-11-10 09:26:08.832722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-10 09:26:31.784704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2859/2859 - 31s - loss: 0.2270 - Accuracy_epochs: 0.9310 - val_loss: 0.1185 - val_Accuracy_epochs: 0.9645 - 31s/epoch - 11ms/step
Epoch 2/5
2859/2859 - 28s - loss: 0.0865 - Accuracy_epochs: 0.9730 - val_loss: 0.0827 - val_Accuracy_epochs: 0.9785 - 28s/epoch - 10ms/step
Epoch 3/5
2859/2859 - 26s - loss: 0.0668 - Accuracy_epochs: 0.9790 - val_loss: 0.0742 - val_Accuracy_epochs: 0.9758 - 26s/epoch - 9ms/step
Epoch 4/5
2859/2859 - 27s - loss: 0.0601 - Accuracy_epochs: 0.9811 - val_loss: 0.0571 - val_Accuracy_epochs: 0.9843 - 27s/epoch - 9ms/step
Epoch 5/5
2859/2859 - 27s - loss: 0.0533 - Accuracy_epochs: 0.9834 - val_loss: 0.0637 - val_Accuracy_epochs: 0.9783 - 27s/epoch - 9ms/step


After the hyperparameter tuning using Tensorboard, the best performing hyperparameter configuration is width = 30, depth = 4 and optimizer = adam with an accuracy of 98.749%. Overall, adam performed better than sgd, and a width of 30 was better than width of 20. Tensorboard made it quite easy to see the result of every trial and make a decision on the best hyperparameters.

It would've been interesting to test larger range of parameters, especially for width and depth, and maybe adding a new hyperparameter to tune, but the current hyperparameter tuning cell already was taking about an hour to run on my computer, so I'll have to test that another time.